In [1]:
import micasense.imageset
import micasense.capture
import micasense.imageset as imageset

import cv2
import os, glob
import json
import pickle #This library will maintain the format as well
from osgeo import gdal,osr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL.Image as Image
import exiftool
import datetime
from math import ceil
from tqdm import tqdm

import algorithms.mutils as mutils

# README

This python notebook details the preprocessing steps needed to prepare the UAV imagery before direct georeferencing (to make things much easier). The preprocessing step is largely based on [Micasense's image processing library]() with some modifications to it.

## Folder organisation
- **flight folder** (a folder that contains all the images captured from a flight, as well as meta data, and processed data used the micasense's image processing library).
- **RawImg** (a folder that contains all the band images captured from a flight e.g. *IMG_0000_1.tif* to *IMG_xxxx_10.tif*. One can easily create a folder named "RawImg" and shift all the raw images here).
    - *IMG* is the prefix for all image captures
    - *_0000* is the index of the image (each capture should have a unique index)
    - *_1* is the band number of the capture, e.g. for a 10-band image, the band number will range from 1 - 10. So each capture will have 10 images e.g. *IMG_0000_1* to *IMG_0000_10*.
- **flight_attributes** (_TO BE CREATED AFTER PREPROCESSING_: a folder that contains the metadata of each capture from the flight).
    - *flight_attributes.csv* a csv file that is automatically generated from the modified micasense preprocessing library
    - *gps_index.txt* a text file that contains the selected gps points selected from the GUI in `algorithms/select_GPS.py`. Run `algorithms/select_GPS.py` to interactively select the start and end points of the flight swaths.
- **stacks** (_TO BE CREATED AFTER PREPROCESSING_: a folder that contains the processed band images, where band images are all band-aligned to create a multispectral image). e.g. the postfix (as a default) always ends with *_1*, but all images in the stacks folder should have a unique image index.
- **thumbnails** (_TO BE CREATED AFTER PREPROCESSING_: a folder that contains the thumbnail rgb of each capture, which allows for fast plotting of rgb orthomosaics)
```
flight folder (*imagePath*)
|
│
└───flight_attributes (*to be created*)
│   │   flight_attributes.csv
│   │   gps_index.txt
│   
└───RawImg
|   │   IMG_0000_1.tif
|   │   IMG_0000_2.tif
|   |   ...
└───stacks (*to be created*)
|   │   IMG_0000_1.tif
|   │   IMG_0001_1.tif
|   |   ...
└───thumbnails (*to be created*)
    │   IMG_0000_1.jpg
    │   IMG_0001_1.jpg
    |   ...
```

In [2]:
panelNames = None
useDLS = True

imagePath =  r"D:\EPMC_flight\16thSur22Nov\F1"

# directory which contains raw images
rawImagePath = os.path.join(imagePath,'RawImg')

# identify the image that has the QR code
panelNames = glob.glob(os.path.join(rawImagePath,'IMG_0000_*.tif'))

outputPath = os.path.join(imagePath,'stacks')
thumbnailPath = os.path.join(imagePath, 'thumbnails')

overwrite = False # Set to False to continue interrupted processing
generateThumbnails = True

# Allow this code to align both radiance and reflectance images; bu excluding
# a definition for panelNames above, radiance images will be used
# For panel images, efforts will be made to automatically extract the panel information
# but if the panel/firmware is before Altum 1.3.5, RedEdge 5.1.7 the panel reflectance
# will need to be set in the panel_reflectance_by_band variable.
# Note: radiance images will not be used to properly create NDVI/NDRE images below.
if panelNames is not None:
    panelCap = micasense.capture.Capture.from_filelist(panelNames)
    warp_matrices = panelCap.get_warp_matrices()
    cropped_dimensions, _ = micasense.imageutils.find_crop_bounds(panelCap, warp_matrices, warp_mode=cv2.MOTION_HOMOGRAPHY)
    print(cropped_dimensions)
else:
    panelCap = None
    warp_matrices = None
if panelCap is not None:
    if panelCap.panel_albedo() is not None:
        panel_reflectance_by_band = panelCap.panel_albedo()
    else:
        panel_reflectance_by_band = [0.65]*len(panelCap.images) #inexact, but quick
    panel_irradiance = panelCap.panel_irradiance(panel_reflectance_by_band)    
    img_type = "reflectance"
else:
    if useDLS:
        img_type='reflectance'
    else:
        img_type = "radiance"

(20.0, 34.0, 1226.0, 919.0)


In [3]:
# batch import images
imgset = imageset.ImageSet.from_directory(rawImagePath)

use_multi_process = True # set to False for single-process saving
overwrite_existing = False # skip existing files, set to True to overwrite

if not os.path.exists(outputPath):
    os.makedirs(outputPath)
if generateThumbnails and not os.path.exists(thumbnailPath):
    os.makedirs(thumbnailPath)

# If we didn't provide a panel above, irradiance set to None will cause DLS data to be used
try:
    irradiance = panel_irradiance+[0]
except NameError:
    irradiance = None

start_time = datetime.datetime.now()

# Save all captures in the imageset as aligned stacks
imgset.process_imageset(outputPath,
                     thumbnailPath,
                     warp_matrices,
                     irradiance = irradiance,
                     img_type='reflectance',
                     multiprocess=use_multi_process, 
                     overwrite=overwrite_existing)

end_time = datetime.datetime.now()

print("Saving time: {}".format(end_time-start_time))
print("Alignment+Saving rate: {:.2f} captures per second".format(float(len(imgset.captures))/float((end_time-start_time).total_seconds())))

Loading ImageSet from: D:\EPMC_flight\16thSur22Nov\F1\RawImg
Processing 477 Captures ...
Processing complete.
Saving time: 0:12:03.705786
Alignment+Saving rate: 0.66 captures per second


# Selecting GPS coordinates

Running the following code will allow an interactive plot to automatically pop-up where users can have the flexibility to select the points along the flight swaths to conduct orthomosaicking. One only needs to select the start and end points of each flight swaths as follows:

![demo of how to use select_GPS.py](plots/selectGPSdemo.gif)

After selecting the GPS points, a file `gps_index.txt` will immediately created in the flight folder, under flight attributes folder:

```
flight folder (*imagePath*)
|
│
└───flight_attributes (*to be created*)
   │   flight_attributes.csv
   │   gps_index.txt
```

In [12]:
%matplotlib qt

In [14]:
from algorithms.select_GPS import draw_plot
imagePath =  r"D:\EPMC_flight\16thSur22Nov\F1"
draw_plot(os.path.join(imagePath,'flight_attributes'))

[]

Coords: [(103.6422191, 1.2333617)]
Indices: [440]
Coords: [(103.6422191, 1.2333617), (103.6422127, 1.2322857)]
Indices: [440, 417]
Coords: [(103.6422191, 1.2333617), (103.6422127, 1.2322857), (103.6423309, 1.2333542)]
Indices: [440, 417, 386]
Coords: [(103.6422191, 1.2333617), (103.6422127, 1.2322857), (103.6423309, 1.2333542), (103.6423251, 1.2322799)]
Indices: [440, 417, 386, 409]
Coords: [(103.6422191, 1.2333617), (103.6422127, 1.2322857), (103.6423309, 1.2333542), (103.6423251, 1.2322799), (103.6424379, 1.2322622)]
Indices: [440, 417, 386, 409, 355]
Coords: [(103.6422191, 1.2333617), (103.6422127, 1.2322857), (103.6423309, 1.2333542), (103.6423251, 1.2322799), (103.6424379, 1.2322622), (103.6424431, 1.2333319)]
Indices: [440, 417, 386, 409, 355, 378]
Coords: [(103.6422191, 1.2333617), (103.6422127, 1.2322857), (103.6423309, 1.2333542), (103.6423251, 1.2322799), (103.6424379, 1.2322622), (103.6424431, 1.2333319), (103.6425523, 1.2333214)]
Indices: [440, 417, 386, 409, 355, 378, 325]